In [12]:
# IPython is what you are using now to run the notebook
import IPython
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [13]:
data_calendar = pd.read_csv('../Data/calendar.csv')

In [15]:
data_calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,21456,2020-02-12,t,$99.00,$99.00,30.0,365.0
1,21456,2020-02-13,t,$99.00,$99.00,30.0,365.0
2,21456,2020-02-14,t,$99.00,$99.00,30.0,365.0
3,21456,2020-02-15,t,$99.00,$99.00,30.0,365.0
4,21456,2020-02-16,t,$99.00,$99.00,30.0,365.0


In [16]:
data = pd.read_csv('../Data/listing_full.csv')

In [17]:
data

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20200212052319,2020-02-12,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2,2,0,0,0.39
1,3831,https://www.airbnb.com/rooms/3831,20200212052319,2020-02-13,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.69
2,5099,https://www.airbnb.com/rooms/5099,20200212052319,2020-02-12,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,none,My neighborhood in Midtown East is called Murr...,...,f,f,moderate,t,t,1,1,0,0,0.59
3,5121,https://www.airbnb.com/rooms/5121,20200212052319,2020-02-12,BlissArtsSpace!,NaN,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,0.38
4,5178,https://www.airbnb.com/rooms/5178,20200212052319,2020-02-13,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"You will use one large, furnished, private roo...",Please don’t expect the luxury here just a bas...,none,"Theater district, many restaurants around here.",...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,3.53


In [3]:
list(data.columns)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [4]:
import numpy as np
import pandas as pd
import re

data_calendar = pd.read_csv('../Data/calendar.csv')
listings_full = pd.read_csv('../Data/listing_full.csv')

# =============================================================================
# calendar
# =============================================================================

na_column = data_calendar.apply(lambda x: sum(x.isnull()), axis = 0)

def num_na(x):
    return sum(x.isnull())
na_by_id = (data_calendar
     .groupby('listing_id')
     .agg(
        num_price_na = ('price', num_na),
        num_adj_price_na = ('adjusted_price', num_na),
        num_min_night_na = ('minimum_nights', num_na),
        num_max_night_na = ('maximum_nights', num_na)
     ))

has_na_listing_id = na_by_id[na_by_id.apply(lambda x: any(x.iloc[0:4]), axis = 1)]

price_na_listing_id = na_by_id[(na_by_id['num_price_na'] > 0) | (na_by_id['num_adj_price_na'] > 0)].index
# five listings have missing data, the number of rows with missing data is around 170, almost half the 365 days,
# therefore, we delete the record of this 5 listings
len(price_na_listing_id)
data_calendar = data_calendar[~data_calendar['listing_id'].isin(price_na_listing_id)]

def to_num(x):
    x = re.sub(r',', '', x)
    return float(x[1:])
data_calendar['price'] = list(map(to_num, data_calendar['price']))
data_calendar['adjusted_price'] = list(map(to_num, data_calendar['adjusted_price']))

calendar_summary = (
        data_calendar
        .groupby('listing_id')
        .agg({
            'available': lambda x: sum(x == 't') / len(x),
            'price': np.mean,
            'adjusted_price': np.mean,
            'minimum_nights': np.mean,
            'maximum_nights': np.mean
        }))

calendar_summary = calendar_summary.reset_index()
calendar_summary.rename(columns = {'price':'price_avrg', 'adjusted_price':'adjusted_price_avrg'}, inplace=True)



# =============================================================================
# Listing
# =============================================================================

percent_NA = listings_full.isnull().sum() * 100 / len(listings_full)
NA_info = pd.DataFrame({'percent_NA': percent_NA})
NA_info[NA_info["percent_NA"]>5]

col_many_na = percent_NA[percent_NA > 60].index
listings_full.drop(col_many_na, axis = 1, inplace = True)

#col_var = {}
#for i in listings_full.columns:
#    col_var[i] = len(np.unique(listings_full[i]))

col_to_del = ['listing_url', 'scrape_id', 'last_scraped', 'experiences_offered',
              'picture_url', 'host_url', 'country_code', 'country', 'calendar_last_scraped']
listing1 = listings_full.drop(col_to_del, axis = 1)


percent_NA = listing1.isnull().sum() * 100 / len(listing1)
col_w_na = percent_NA[percent_NA > 0].index

# =============================================================================
# numeric variable
# =============================================================================

# convert $ into number
def to_num(x):
    x = re.sub(r',', '', x)
    return float(x[1:])

listing1['price'] = list(map(to_num, listing1['price']))
valid_ind_security = ~listing1['security_deposit'].isnull()
listing1.loc[valid_ind_security, 'security_deposit'] = list(
        map(to_num, listing1.loc[valid_ind_security, 'security_deposit']))
valid_ind_clean = ~listing1['cleaning_fee'].isnull()
listing1.loc[valid_ind_clean, 'cleaning_fee'] = list(
        map(to_num, listing1.loc[valid_ind_clean, 'cleaning_fee']))
listing1['extra_people'] = list(map(to_num, listing1['extra_people']))

def str_to_pct(x):
    x = re.sub(r'%', '', x)
    return float(x) / 100
valid_ind_host= ~listing1['host_response_rate'].isnull()
listing1.loc[valid_ind_host, 'host_response_rate'] = list(
        map(str_to_pct, listing1.loc[valid_ind_host, 'host_response_rate']))

# numeric col with na
num_col_w_na = [
        'host_response_rate', 'host_listings_count', 'host_total_listings_count',
        'bathrooms', 'bedrooms', 'beds', 'security_deposit',
        'cleaning_fee', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'reviews_per_month']
mean_na_col = {}
for col in num_col_w_na:
    mean_na_col[col] = np.mean(listing1[col])

for col_name in num_col_w_na:
    na_row_ind = listing1[col_name].isnull()
    listing1.loc[na_row_ind, col_name] = mean_na_col[col_name]

# =============================================================================
# Qualitative Variable
# =============================================================================
#cat_col = ['host_response_time', 'host_neighbourhood', 'host_has_profile_pic',
#           'host_is_superhost', 'host_identity_verified', 'neighbourhood',
#           'city', 'state', 'market']
qualtv_col = col_w_na[~col_w_na.isin(num_col_w_na)]
for col_name in qualtv_col:
    na_row_ind = listing1[col_name].isnull()
    listing1.loc[na_row_ind, col_name] = 'Unknown'

# for columns of word description, i.e. non-categorical variables,
# convert them into binary variable of having content vs. not
col_to_binary = ['name', 'summary', 'space', 'description', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules', 'host_name',
       'host_location', 'host_about', 'host_thumbnail_url',
       'host_picture_url', 'zipcode', 'first_review', 'last_review']
for col in col_to_binary:
    binary_col_name = 'has_' + col
    listing1[binary_col_name] = (listing1[col] != 'Unknown').astype(int)

#listing1.to_csv("listing_clean.csv", index = False)





# =============================================================================
# combine
# =============================================================================

listing1.shape
calendar_summary.shape


sum(listing1['id'].isin(calendar_summary['listing_id']))
sum(calendar_summary['listing_id'].isin(listing1['id']))

listing_calendar = pd.merge(listing1, calendar_summary, 'inner',
                            left_on = 'id', right_on = 'listing_id')
listing_calendar.drop('listing_id', axis = 1, inplace = True)

listing_calendar.head()

,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,...,has_host_thumbnail_url,has_host_picture_url,has_zipcode,has_first_review,has_last_review,available,price_avrg,adjusted_price_avrg,minimum_nights_y,maximum_nights_y
0,2595,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,Unknown,Apartment is located on 37th Street between 5t...,"Guests have full access to the kitchen, bathro...",I am a Sound Therapy Practitioner and Kundalin...,...,1,1,1,1,1,1.000000,175.000000,175.000000,7.000000,1125.0
1,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Just the right mix of urban center and local n...,Unknown,B52 bus for a 10-minute ride to downtown Brook...,"You will have the private, exclusive use of an...","We'll be around, but since you have the top fl...",...,1,1,1,1,1,0.375342,82.934247,82.934247,1.000000,730.0
2,5099,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,My neighborhood in Midtown East is called Murr...,Read My Full Listing For All Information. New ...,From the apartment is a 10 minute walk to Gran...,I will meet you upon arrival.,I usually check in with guests via text or ema...,...,1,1,1,1,1,0.000000,200.000000,200.000000,3.000000,21.0
3,5121,BlissArtsSpace!,Unknown,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,Unknown,Unknown,Unknown,Unknown,Unknown,...,1,1,1,1,1,1.000000,60.000000,60.000000,29.000000,730.0
4,5178,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"You will use one large, furnished, private roo...",Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",Reservation should be made at least a few days...,Unknown,Bathroom is shared with the host but the kitch...,Unknown,...,1,1,1,1,1,0.591781,78.172603,78.172603,1.868493,14.0


In [5]:
listing_calendar.room_type.unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [6]:
is_shared =  listing_calendar['room_type']== "Private room"
df_shared = listing_calendar[is_shared]
df_shared.bedrooms.unique()

array([ 1.        ,  4.        ,  3.        ,  2.        ,  0.        ,
        9.        ,  5.        ,  6.        ,  8.        ,  1.17992239,
       11.        ])

In [15]:
data = pd.read_csv('../Data/listing_calendar_clean.csv')

/Users/yihenghe/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,11,17,23,53,54,62,63,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
data.head()

,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,...,has_host_thumbnail_url,has_host_picture_url,has_zipcode,has_first_review,has_last_review,available,price_avrg,adjusted_price_avrg,minimum_nights_y,maximum_nights_y
0,2595,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,Unknown,Apartment is located on 37th Street between 5t...,"Guests have full access to the kitchen, bathro...",I am a Sound Therapy Practitioner and Kundalin...,...,0.0,0.0,0.0,0.0,0.0,0.928962,196.415301,196.415301,10.000000,1125.0
1,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Just the right mix of urban center and local n...,Unknown,B52 bus for a 10-minute ride to downtown Brook...,You will have exclusive use of and access to: ...,"We'll be around, but since you have the top fl...",...,0.0,0.0,0.0,0.0,0.0,0.445355,86.221311,86.221311,1.000000,730.0
2,5099,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,My neighborhood in Midtown East is called Murr...,Read My Full Listing For All Information. New ...,From the apartment is a 10 minute walk to Gran...,I will meet you upon arrival.,I usually check in with guests via text or ema...,...,0.0,0.0,0.0,0.0,0.0,0.052055,206.164384,206.164384,3.000000,21.0
3,5121,BlissArtsSpace!,Unknown,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,Unknown,Unknown,Unknown,Unknown,Unknown,...,0.0,0.0,0.0,0.0,0.0,1.000000,60.000000,60.000000,29.000000,730.0
4,5178,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"You will use one large, furnished, private roo...",Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",Reservation should be made at least a few days...,Unknown,Bathroom is shared with the host but the kitch...,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.663014,78.279452,78.279452,1.750685,14.0


In [181]:
# select categorical independent variables
X_factor = data[["host_response_time","host_is_superhost","host_neighbourhood","host_has_profile_pic", 
                 "host_identity_verified", "neighbourhood_group_cleansed","zipcode","is_location_exact",
                "property_type", "room_type", "bed_type", "has_availability", "requires_license",
                "instant_bookable", "is_business_travel_ready", "cancellation_policy", 
                 "require_guest_profile_picture", "require_guest_phone_verification",
                "has_name", "has_summary", "has_space", "has_description", "has_neighborhood_overview",
                "has_notes", "has_transit", "has_access","has_interaction","has_house_rules", 
                 "has_host_name", "has_host_location", "has_host_about", "has_host_thumbnail_url",
                "has_host_picture_url", "has_zipcode", "has_first_review", "has_last_review"]]

In [182]:
X_factor.shape

(50596, 36)

In [183]:
# select quantitive independent variables
X_quantitive = data[["host_response_rate","host_total_listings_count","accommodates","bathrooms",
                    "bedrooms", "beds", "security_deposit", "cleaning_fee", "guests_included",
                    "extra_people", "minimum_nights_avg_ntm","maximum_nights_avg_ntm", "availability_30",
                     "availability_60","availability_90","availability_365", "number_of_reviews",
                    "number_of_reviews_ltm","review_scores_rating", "review_scores_accuracy",
                    "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication",
                    "review_scores_location", "review_scores_value", "calculated_host_listings_count",
                    "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms",
                    "calculated_host_listings_count_shared_rooms", "reviews_per_month", "available"]]

In [184]:
X_quantitive.loc[X_quantitive['host_response_rate'] == "NY"]

,host_response_rate,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,security_deposit,cleaning_fee,guests_included,extra_people,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,available
7931,NY,t,1125.0,1125.0,1.0,1125.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
16045,NY,t,21.0,21.0,1.0,21.0,0.0,0.0,0.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [185]:
X_quantitive = X_quantitive.drop(X_quantitive.index[[7931, 16045]])

In [188]:
X_quantitive.loc[X_quantitive['host_response_rate'] == "NY"]

(50594, 31)

In [189]:
X_quantitive.shape

(50594, 31)

In [190]:
if 'moderate' in list(X_quantitive["availability_365"]):
    print('NY in')
else:
    print('NY not')

NY not


In [199]:
# standardize all quantitive independent variables for model fitting
X_quantitive_scaled = preprocessing.scale(X_quantitive)
X_quantitive_scaled = pd.DataFrame(data=X_quantitive_scaled, columns=["host_response_rate","host_total_listings_count","accommodates","bathrooms",
                    "bedrooms", "beds", "security_deposit", "cleaning_fee", "guests_included",
                    "extra_people", "minimum_nights_avg_ntm","maximum_nights_avg_ntm", "availability_30",
                     "availability_60","availability_90","availability_365", "number_of_reviews",
                    "number_of_reviews_ltm","review_scores_rating", "review_scores_accuracy",
                    "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication",
                    "review_scores_location", "review_scores_value", "calculated_host_listings_count",
                    "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms",
                    "calculated_host_listings_count_shared_rooms", "reviews_per_month", "available"])
X_quantitive_scaled.head()

,host_response_rate,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,security_deposit,cleaning_fee,guests_included,extra_people,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,available
0,-0.626004,-0.102627,-0.978825,-0.340208,-1.553241,-0.486408,0.193055,0.543426,-0.448269,-0.589574,...,0.40117,0.386821,0.626514,-0.467786,-0.193258,-0.142864,-0.227835,-0.125595,-0.670694,1.587365
1,0.506493,-0.147698,0.073285,-0.340208,-0.238298,2.168545,0.552625,-0.000014,-0.448269,-0.589574,...,0.40117,-1.121541,0.626514,-0.467786,-0.193258,-0.142864,-0.227835,-0.125595,2.155076,0.299828
2,0.000009,-0.147698,-0.452770,-0.340208,-0.238298,-0.486408,0.073199,1.127593,0.411852,3.505791,...,0.40117,0.386821,0.626514,-0.467786,-0.193258,-0.142864,-0.227835,-0.125595,-0.532046,-0.747282
3,0.506493,-0.147698,-0.452770,-0.000002,-0.238298,-0.486408,0.432769,-1.306435,-0.448269,0.639035,...,0.40117,0.386821,-0.867228,-0.467786,-0.193258,-0.173855,-0.073906,-0.125595,-0.677296,1.776495
4,0.506493,-0.147698,-0.452770,-0.340208,-0.238298,-0.486408,0.000020,-1.014352,-0.448269,-0.098130,...,-1.13975,-1.121541,0.626514,-1.688277,-0.193258,-0.173855,-0.073906,-0.125595,1.395816,0.879314


In [192]:
# remove row 7931 and 16045
X_factor = X_factor.drop(X_factor.index[[7931, 16045]])
X_factor.shape

(50594, 36)

In [193]:
# combine independent variables
X_scaled = pd.concat([X_quantitive_scaled, X_factor], axis=1)

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [178]:
Y = data["price_avrg"]